# Run necessary background code

In [1]:
execfile('import_packages.py')

In [4]:
execfile('read_in_data.py')

C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Functions and base values

In [22]:
execfile('amt_model.py')
execfile('ftc_model.py')
execfile('sec199_model.py')
execfile('ccr_model.py')
#execfile('interest_model.py')

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
{'tau_scg': 0.319856951254272, 'tau_lcg': 0.22569175423344318, 'tau_h': 0.1716527226840607, 'tau_nc': 0.3205428658733798, 'tau_int': 0.3460716542050802, 'tau_div': 0.19227573879552445, 'tau_td': 0.24568244687095886}


C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\si

params1960
params1961
params1962
params1963
params1964
params1965
params1966
params1967
params1968
params1969
params1970
params1971
params1972
params1973
params1974
params1975
params1976
params1977
params1978
params1979
params1980
params1981
params1982
params1983
params1984
params1985
params1986
params1987
params1988
params1989
params1990
params1991
params1992
params1993
params1994
params1995
params1996
params1997
params1998
params1999
params2000
params2001


C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\si

params2002
params2003
params2004
params2005
params2006
params2007
params2008
params2009
params2010
params2011
params2012
params2013
params2014
params2015
params2016


C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\si

params2017
params2018
params2019
params2020
params2021
params2022
params2023
params2024
params2025
params2026
params2027
params2028
params2029
params2030
params2031
params2032
params2033
params2034
params1960
params1961
params1962
params1963
params1964
params1965
params1966
params1967
params1968
params1969
params1970
params1971
params1972
params1973
params1974
params1975
params1976
params1977
params1978
params1979
params1980
params1981
params1982
params1983
params1984
params1985
params1986
params1987
params1988
params1989
params1990
params1991
params1992
params1993
params1994
params1995
params1996
params1997
params1998
params1999
params2000
params2001
params2002
params2003
params2004
params2005
params2006
params2007
params2008
params2009
params2010
params2011
params2012
params2013
params2014
params2015
params2016
params2017
params2018
params2019
params2020
params2021
params2022
params2023
params2024
params2025
params2026
params2027
params2028
params2029
params2030
params2031
params2032

In [23]:
execfile('gen_baseline.py')
execfile('usercode_taxcalc.py')

C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


params1960
params1961
params1962
params1963
params1964
params1965
params1966
params1967
params1968
params1969
params1970
params1971
params1972
params1973
params1974
params1975
params1976
params1977
params1978
params1979
params1980
params1981
params1982
params1983
params1984
params1985
params1986
params1987
params1988
params1989
params1990
params1991
params1992
params1993
params1994
params1995
params1996
params1997
params1998
params1999
params2000
params2001


C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\si

params2002
params2003
params2004
params2005
params2006
params2007
params2008
params2009
params2010
params2011
params2012
params2013
params2014
params2015
params2016


C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\si

params2017
params2018
params2019
params2020
params2021
params2022
params2023
params2024
params2025
params2026
params2027
params2028
params2029
params2030
params2031
params2032
params2033
params2034
Depreciation step corp
params1960
params1961
params1962
params1963
params1964
params1965
params1966
params1967
params1968
params1969
params1970
params1971
params1972
params1973
params1974
params1975
params1976
params1977
params1978
params1979
params1980
params1981
params1982
params1983
params1984
params1985
params1986
params1987
params1988
params1989
params1990
params1991
params1992
params1993
params1994
params1995
params1996
params1997
params1998
params1999
params2000
params2001
params2002
params2003
params2004
params2005
params2006
params2007
params2008
params2009
params2010
params2011
params2012
params2013
params2014
params2015
params2016
params2017
params2018
params2019
params2020
params2021
params2022
params2023
params2024
params2025
params2026
params2027
params2028
params2029
params203

In [24]:
capPath_base_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1096.745257,6607.852523,1180.422897,9095.834243,509.468504,852.349826,2014
1,1051.439909,7216.485871,1166.490371,9795.778435,583.040796,897.644661,2015
2,947.054358,7776.698446,976.971527,10503.231537,640.753668,843.326732,2016
3,981.927234,8221.036616,1012.946027,11024.560178,679.208401,883.107302,2017
4,1022.993630,8688.283348,1055.309699,11577.538828,717.811550,835.985755,2018
5,1060.047968,9172.777226,1093.534572,12152.599095,756.873268,815.028863,2019
6,1096.603976,9667.985029,1131.245374,12742.359133,795.723471,646.635494,2020
7,1137.909367,10171.879412,1173.855591,13344.209370,834.357504,734.298078,2021
8,1182.881038,10686.049684,1220.247903,13958.831279,873.366016,809.978905,2022
9,1229.851110,11215.553778,1268.701746,14591.928622,913.400863,872.497447,2023


In [25]:
capPath_base_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,279.639637,2646.084107,315.736931,6942.434842,164.247247,303.839436,2014
1,295.975458,2769.673116,348.397371,7114.980726,175.441735,336.388074,2015
2,301.678613,2924.736946,316.261692,7375.007367,187.772158,330.904854,2016
3,312.787163,3090.535856,327.907227,7631.637926,198.986014,349.171309,2017
4,325.868623,3266.188009,341.621041,7910.355610,210.295484,335.671865,2018
5,337.672065,3449.186732,353.995059,8202.016281,221.771419,330.177732,2019
6,349.316767,3637.334979,366.202664,8503.135876,233.224306,272.222952,2020
7,362.474357,3829.865210,379.996289,8811.986908,244.649658,304.382284,2021
8,376.799819,4027.061932,395.014241,9127.228105,256.206725,332.363018,2022
9,391.761860,4230.637464,410.699544,9451.421877,268.078873,355.857022,2023


In [26]:
combined_base

,year,taxrev,amt,pymtc,ftc,taxbc,gbc_adj,tau,taxinc,sec199,taxDep,nid,ebitda
0,2014,320.731,4.457941,3.700228,101.582376,421.555664,0.02574,0.347,1312.193118,34.306208,852.349826,231.441998,2430.291150
1,2015,343.797,4.529305,3.799722,91.794692,434.862110,0.02574,0.347,1353.612624,34.855388,897.644661,243.981131,2530.093803
2,2016,299.571,4.524316,3.895522,96.318143,395.260349,0.02574,0.347,1230.342644,34.816996,843.326732,248.001094,2356.487465
3,2017,310.006,4.541886,3.978088,99.935946,409.378148,0.02574,0.347,1274.287681,34.952204,883.107302,268.262930,2460.610117
4,2018,324.343,4.550562,4.052119,96.600049,420.444606,0.02574,0.347,1308.734686,35.018974,835.985755,295.580925,2475.320340
5,2019,343.899,4.593510,4.117569,97.986091,441.409149,0.02574,0.347,1373.991854,35.349483,815.028863,328.934835,2553.305035
6,2020,380.350,4.659451,4.180064,103.553749,483.424362,0.02574,0.347,1504.774282,35.856932,646.635494,362.449185,2549.715893
7,2021,382.662,4.755976,4.243011,109.003947,491.152982,0.02574,0.347,1528.831464,36.599743,734.298078,393.699349,2693.428635
8,2022,389.490,4.886556,4.310368,114.806528,503.720339,0.02574,0.347,1567.950378,37.604625,809.978905,422.521900,2838.055807
9,2023,395.128,5.064423,4.386026,121.619274,516.068877,0.02574,0.347,1606.388166,38.973402,872.497447,448.489428,2966.348443


# Apply partial equilibrium responses

## Specify economic and tax parameters

In [59]:
## Economic assumptions
p = 0.2     #real financial return
f = 0.32    #debt financing share
i_d = 0.068   #nominal interest rate on debt
pi = 0.024  #inflation rate
E = 0.058   #real return on equity

## Tax policy parameters
reform_start_year = 2017
tau_base = 0.35     #statutory tax rate, current law
tau_ref = 0.36      #statutory tax rate
progressive_rate = True
int_hc = 0.0        #haircut on interest deduction
## Modify bonus depreciation dataset to use new laws
bonus_data_ref = copy.deepcopy(bonus_data)
#for i in range(len(bonus_data_ref)):
#    if (bonus_data_ref['year'][i] >= reform_start_year):# and (bonus_data_ref['year'][i] < reform_start_year+5):
#        bonus_data_ref['bonus3'][i] = 1.
#        bonus_data_ref['bonus5'][i] = 1.
#        bonus_data_ref['bonus7'][i] = 1.
#        bonus_data_ref['bonus10'][i] = 1.
#        bonus_data_ref['bonus15'][i] = 1.
#        bonus_data_ref['bonus20'][i] = 1.
#        bonus_data_ref['bonus25'][i] = 1.
#        bonus_data_ref['bonus27'][i] = 1.
#        bonus_data_ref['bonus39'][i] = 1.

inventory_accounting = 0 #0 for historical mix of FIFO and LIFO, 1 for FIFO, 2 for LIFO, 3 for expensing
bonusrate_land = 0 #bonus depreciation rate for land. Does not affect model results

## Calculate the investment response

In [60]:
# Set all elasticities
#     inv_usercost_c: elasticity of corporate investment w.r.t. the user cost of capital
#     inv_usercost_nc: elasticity of noncorporate investment w.r.t. the user cost of capital
#     inv_eatr: semi-elasticity of investment w.r.t. the effective average tax rate
#     debt_taxshield_c: semi-elasticity of the corporate debt/asset ratio w.r.t. the marginal tax shield, tau * (1 - hc)
#     debt_taxshield_nc: semi-elasticity of the noncorporate debt/asset ratio w.r.t. the marginal tax shield, tau * (1 - hc)
#     legalform_ratediff: semi-elasticity of the corporate share of business activity w.r.t. the (Tc - Tnc) tax rate differential


elast_dict = {
    'inv_usercost_c': 0.0,
    'inv_usercost_nc': 0.0,
    'inv_eatr': 0.0,
    'debt_taxshield_c': 0.0,
    'debt_taxshield_nc': 0.0,
    'legalform_ratediff': 0.0
}
execfile('responses.py')

In [61]:
response_results = investmentResponse(reform_start_year)

***** RUN 2014 *****
***** RUN 2015 *****
***** RUN 2016 *****
***** RUN 2017 *****
***** RUN 2018 *****
***** RUN 2019 *****
***** RUN 2020 *****
***** RUN 2021 *****
***** RUN 2022 *****
***** RUN 2023 *****
***** RUN 2024 *****
***** RUN 2025 *****
***** RUN 2026 *****
***** RUN 2027 *****


In [62]:
response_results

,Asset,z_c,z_nc,deltaIc2014,deltaEc2014,deltaInc2014,deltaEnc2014,deltaIc2015,deltaEc2015,deltaInc2015,...,deltaInc2025,deltaEnc2025,deltaIc2026,deltaEc2026,deltaInc2026,deltaEnc2026,deltaIc2027,deltaEc2027,deltaInc2027,deltaEnc2027
0,Aerospace products and parts manufacturing,1.000000,1.000000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Air transportation,0.335802,0.349692,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Aircraft,0.935556,0.938395,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"All other nonmanufacturing, n.e.c.",1.000000,1.000000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Amusement and recreation,0.913574,0.917230,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Autos,0.935556,0.938395,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Books,0.862484,0.870057,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Chemical manufacturing, ex. pharma and med",1.000000,1.000000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Communication,0.822237,0.828521,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Communications,0.913574,0.917230,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
## Create new investment matrices for corporate and noncorporate
(inv_mat_ref_corp, inv_mat_ref_noncorp) = buildNewInvMatrix(response_results, 'usercost')

In [64]:
## Redo capital calculations with new investment matrices
annualDepreciation_ref_corp = annualCCRdeduction(inv_mat_ref_corp, btax_defaults, adjfactor_dep_corp)
annualDepreciation_ref_noncorp = annualCCRdeduction(inv_mat_ref_noncorp, btax_defaults, adjfactor_dep_noncorp)

C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


params1960
params1961
params1962
params1963
params1964
params1965
params1966
params1967
params1968
params1969
params1970
params1971
params1972
params1973
params1974
params1975
params1976
params1977
params1978
params1979
params1980
params1981
params1982
params1983
params1984
params1985
params1986
params1987
params1988
params1989
params1990
params1991
params1992
params1993
params1994
params1995
params1996
params1997
params1998
params1999
params2000
params2001


C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\si

params2002
params2003
params2004
params2005
params2006
params2007
params2008
params2009
params2010
params2011
params2012
params2013
params2014
params2015
params2016


C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\si

params2017
params2018
params2019
params2020
params2021
params2022
params2023
params2024
params2025
params2026
params2027
params2028
params2029
params2030
params2031
params2032
params2033
params2034
params1960
params1961
params1962
params1963
params1964
params1965
params1966
params1967
params1968
params1969
params1970
params1971
params1972
params1973
params1974
params1975
params1976
params1977
params1978
params1979
params1980
params1981
params1982
params1983
params1984
params1985
params1986
params1987
params1988
params1989
params1990
params1991
params1992
params1993
params1994
params1995
params1996
params1997
params1998
params1999
params2000
params2001
params2002
params2003
params2004
params2005
params2006
params2007
params2008
params2009
params2010
params2011
params2012
params2013
params2014
params2015
params2016
params2017
params2018
params2019
params2020
params2021
params2022
params2023
params2024
params2025
params2026
params2027
params2028
params2029
params2030
params2031
params2032

In [65]:
## Generate new capital stock path
(capPath_ref_corp, taxDep_ref_corp) = capitalPath(inv_mat_ref_corp, annualDepreciation_ref_corp)
(capPath_ref_noncorp, taxDep_ref_noncorp) = capitalPath(inv_mat_ref_noncorp, annualDepreciation_ref_noncorp, corp_noncorp=False)

In [66]:
capPath_ref_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1096.745257,6607.852523,1180.422897,9095.834243,509.468504,852.349826,2014
1,1051.439909,7216.485871,1166.490371,9795.778435,583.040796,897.644661,2015
2,947.054358,7776.698446,976.971527,10503.231537,640.753668,843.326732,2016
3,981.927234,8221.036616,1012.946027,11024.560178,679.208401,883.107302,2017
4,1022.993630,8688.283348,1055.309699,11577.538828,717.811550,835.985755,2018
5,1060.047968,9172.777226,1093.534572,12152.599095,756.873268,815.028863,2019
6,1096.603976,9667.985029,1131.245374,12742.359133,795.723471,646.635494,2020
7,1137.909367,10171.879412,1173.855591,13344.209370,834.357504,734.298078,2021
8,1182.881038,10686.049684,1220.247903,13958.831279,873.366016,809.978905,2022
9,1229.851110,11215.553778,1268.701746,14591.928622,913.400863,872.497447,2023


In [67]:
capPath_base_corp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,1096.745257,6607.852523,1180.422897,9095.834243,509.468504,852.349826,2014
1,1051.439909,7216.485871,1166.490371,9795.778435,583.040796,897.644661,2015
2,947.054358,7776.698446,976.971527,10503.231537,640.753668,843.326732,2016
3,981.927234,8221.036616,1012.946027,11024.560178,679.208401,883.107302,2017
4,1022.993630,8688.283348,1055.309699,11577.538828,717.811550,835.985755,2018
5,1060.047968,9172.777226,1093.534572,12152.599095,756.873268,815.028863,2019
6,1096.603976,9667.985029,1131.245374,12742.359133,795.723471,646.635494,2020
7,1137.909367,10171.879412,1173.855591,13344.209370,834.357504,734.298078,2021
8,1182.881038,10686.049684,1220.247903,13958.831279,873.366016,809.978905,2022
9,1229.851110,11215.553778,1268.701746,14591.928622,913.400863,872.497447,2023


In [68]:
capPath_ref_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,279.639637,2646.084107,315.736931,6942.434842,164.247247,303.839436,2014
1,295.975458,2769.673116,348.397371,7114.980726,175.441735,336.388074,2015
2,301.678613,2924.736946,316.261692,7375.007367,187.772158,330.904854,2016
3,312.787163,3090.535856,327.907227,7631.637926,198.986014,349.171309,2017
4,325.868623,3266.188009,341.621041,7910.355610,210.295484,335.671865,2018
5,337.672065,3449.186732,353.995059,8202.016281,221.771419,330.177732,2019
6,349.316767,3637.334979,366.202664,8503.135876,233.224306,272.222952,2020
7,362.474357,3829.865210,379.996289,8811.986908,244.649658,304.382284,2021
8,376.799819,4027.061932,395.014241,9127.228105,256.206725,332.363018,2022
9,391.761860,4230.637464,410.699544,9451.421877,268.078873,355.857022,2023


In [69]:
capPath_base_noncorp

,FixedInv,FixedK,Investment,Kstock,TrueDep,taxDep,year
0,279.639637,2646.084107,315.736931,6942.434842,164.247247,303.839436,2014
1,295.975458,2769.673116,348.397371,7114.980726,175.441735,336.388074,2015
2,301.678613,2924.736946,316.261692,7375.007367,187.772158,330.904854,2016
3,312.787163,3090.535856,327.907227,7631.637926,198.986014,349.171309,2017
4,325.868623,3266.188009,341.621041,7910.355610,210.295484,335.671865,2018
5,337.672065,3449.186732,353.995059,8202.016281,221.771419,330.177732,2019
6,349.316767,3637.334979,366.202664,8503.135876,233.224306,272.222952,2020
7,362.474357,3829.865210,379.996289,8811.986908,244.649658,304.382284,2021
8,376.799819,4027.061932,395.014241,9127.228105,256.206725,332.363018,2022
9,391.761860,4230.637464,410.699544,9451.421877,268.078873,355.857022,2023


## Change in earnings

In [80]:
earnings_ref_data = earningsResponse(response_results)
earnings_ref_data['earnings_base'] = combined_base['ebitda'].tolist()
earnings_ref_data['ebitda'] = earnings_ref_data['earnings_base'] + earnings_ref_data['deltaE']
#earnings_ref_data.drop(['deltaE', 'earnings_base'], axis=1, inplace=True)
combined_ref = earnings_ref_data.merge(right=taxDep_ref_corp, how='outer', on='year')

In [71]:
earnings_ref_data

,deltaE,year,earnings_base,ebitda
0,0.0,2014,2430.291150,2430.291150
1,0.0,2015,2530.093803,2530.093803
2,0.0,2016,2356.487465,2356.487465
3,0.0,2017,2460.610117,2460.610117
4,0.0,2018,2475.320340,2475.320340
5,0.0,2019,2553.305035,2553.305035
6,0.0,2020,2549.715893,2549.715893
7,0.0,2021,2693.428635,2693.428635
8,0.0,2022,2838.055807,2838.055807
9,0.0,2023,2966.348443,2966.348443


# Recalculate corporate tax using reform parameters

##  NID

In [72]:
iit_params_ref = {
    reform_start_year: {
        '_II_rt7': [0.396]
    }
}

In [81]:
(NID_ref, NIP_ref) = NID_response(capPath_ref_corp)
(IntDed_ref_noncorp, IntPaid_ref_noncorp) = noncorpIntDeduction_response(capPath_base_noncorp, id_hc_year=reform_start_year,
                                                                         id_hc_old=int_hc, id_hc_new=int_hc)
combined_ref = combined_ref.merge(right=NID_ref, how='outer', on='year')

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


## Section 199

In [82]:
sec199_reform = sec199()
combined_ref = combined_ref.merge(right=sec199_reform, how='outer', on='year')

## Taxable income and tax before credits

In [83]:
combined_ref['taxinc'] = combined_ref['ebitda'] - combined_ref['taxDep'] - combined_ref['nid'] - combined_ref['sec199']
combined_ref['gbc_adj'] = 0.025739617
combined_ref['tau'] = 0.347
combined_ref['tau'][reform_start_year-2014:] = tau_ref - progressive_rate * 0.003
combined_ref['taxbc'] = combined_ref['taxinc'] * (combined_ref['tau'] - combined_ref['gbc_adj'])

C:\Users\cody.kallen\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [84]:
combined_ref = combined_ref.merge(right=ftc_base, how='outer', on='year')
amt_ref = AMTmodel(amt_repeal_year=9e99, ctax_rates=np.asarray(combined_ref['tau']))
combined_ref = combined_ref.merge(right=amt_ref, how='outer', on='year')
combined_ref['taxrev'] = combined_ref['taxbc'] + combined_ref['amt'] - combined_ref['ftc'] - combined_ref['pymtc']
combined_ref

,deltaE,year,earnings_base,ebitda,taxDep,nid,sec199,taxinc,gbc_adj,tau,taxbc,ftc,amt,pymtc,taxrev
0,0.0,2014,2430.291150,2430.291150,852.349826,231.441998,34.306208,1312.193118,0.02574,0.347,421.555664,101.582376,4.457941,3.700228,320.731000
1,0.0,2015,2530.093803,2530.093803,897.644661,243.981131,34.855388,1353.612624,0.02574,0.347,434.862110,91.794692,4.529305,3.799722,343.797000
2,0.0,2016,2356.487465,2356.487465,843.326732,248.001094,34.816996,1230.342644,0.02574,0.347,395.260349,96.318143,4.524316,3.895522,299.571000
3,0.0,2017,2460.610117,2460.610117,883.107302,268.262930,34.952204,1274.287681,0.02574,0.357,422.121025,99.935946,4.414662,3.978088,322.621653
4,0.0,2018,2475.320340,2475.320340,835.985755,295.580925,35.018974,1308.734686,0.02574,0.357,433.531953,96.600049,4.423095,4.035414,337.319586
5,0.0,2019,2553.305035,2553.305035,815.028863,328.934835,35.349483,1373.991854,0.02574,0.357,455.149068,97.986091,4.464841,4.086319,357.541498
6,0.0,2020,2549.715893,2549.715893,646.635494,362.449185,35.856932,1504.774282,0.02574,0.357,498.472105,103.553749,4.528934,4.136022,395.311267
7,0.0,2021,2693.428635,2693.428635,734.298078,393.699349,36.599743,1528.831464,0.02574,0.357,506.441296,109.003947,4.622756,4.187615,397.872490
8,0.0,2022,2838.055807,2838.055807,809.978905,422.521900,37.604625,1567.950378,0.02574,0.357,519.399843,114.806528,4.749678,4.244752,405.098241
9,0.0,2023,2966.348443,2966.348443,872.497447,448.489428,38.973402,1606.388166,0.02574,0.357,532.132759,121.619274,4.922562,4.311053,411.124994


In [85]:
combined_base

,year,taxrev,amt,pymtc,ftc,taxbc,gbc_adj,tau,taxinc,sec199,taxDep,nid,ebitda
0,2014,320.731,4.457941,3.700228,101.582376,421.555664,0.02574,0.347,1312.193118,34.306208,852.349826,231.441998,2430.291150
1,2015,343.797,4.529305,3.799722,91.794692,434.862110,0.02574,0.347,1353.612624,34.855388,897.644661,243.981131,2530.093803
2,2016,299.571,4.524316,3.895522,96.318143,395.260349,0.02574,0.347,1230.342644,34.816996,843.326732,248.001094,2356.487465
3,2017,310.006,4.541886,3.978088,99.935946,409.378148,0.02574,0.347,1274.287681,34.952204,883.107302,268.262930,2460.610117
4,2018,324.343,4.550562,4.052119,96.600049,420.444606,0.02574,0.347,1308.734686,35.018974,835.985755,295.580925,2475.320340
5,2019,343.899,4.593510,4.117569,97.986091,441.409149,0.02574,0.347,1373.991854,35.349483,815.028863,328.934835,2553.305035
6,2020,380.350,4.659451,4.180064,103.553749,483.424362,0.02574,0.347,1504.774282,35.856932,646.635494,362.449185,2549.715893
7,2021,382.662,4.755976,4.243011,109.003947,491.152982,0.02574,0.347,1528.831464,36.599743,734.298078,393.699349,2693.428635
8,2022,389.490,4.886556,4.310368,114.806528,503.720339,0.02574,0.347,1567.950378,37.604625,809.978905,422.521900,2838.055807
9,2023,395.128,5.064423,4.386026,121.619274,516.068877,0.02574,0.347,1606.388166,38.973402,872.497447,448.489428,2966.348443


In [86]:
compare = combined_ref['taxrev'] - combined_base['taxrev']
print ("Change in corporate tax revenue, " + str(reform_start_year) + "-" + str(reform_start_year + 9) + ": " +
       str(int((sum(combined_ref['taxrev'][reform_start_year-2014:reform_start_year-2004]) - 
           sum(combined_base['taxrev'][reform_start_year-2014:reform_start_year-2004])) * 100.) / 100.) + 
       " billion dollars")

Change in corporate tax revenue, 2017-2026: 152.12 billion dollars


In [87]:
for i in compare:
    print i

0.0
5.68434188608e-14
-1.13686837722e-13
12.6156531206
12.9765856299
13.642498121
14.9612673999
15.210490358
15.6082407456
15.9969937776
16.489015185
17.0231112906
17.5967406854
18.2325160079


## Calculate percent changes in investment income and pass-through income

In [88]:
execfile('multipliers_for_taxcalc.py')

In [89]:
indiv_gfactors

,Investment,year,inc_aftertax_base,inc_aftertax_ref,equity,debt,SchC_pos,SchC_neg,e26270_pos,e26270_neg
0,1180.422897,2014,1368.649648,1368.649648,1.000000,1.0,1.0,1.0,1.0,1.0
1,1166.490371,2015,1359.274877,1359.274877,1.000000,1.0,1.0,1.0,1.0,1.0
2,976.971527,2016,1168.161704,1168.161704,1.000000,1.0,1.0,1.0,1.0,1.0
3,1012.946027,2017,1203.132786,1190.517133,0.989514,1.0,1.0,1.0,1.0,1.0
4,1055.309699,2018,1137.584865,1124.608279,0.988593,1.0,1.0,1.0,1.0,1.0
5,1093.534572,2019,1123.597932,1109.955434,0.987858,1.0,1.0,1.0,1.0,1.0
6,1131.245374,2020,1011.193237,996.231970,0.985204,1.0,1.0,1.0,1.0,1.0
7,1173.855591,2021,1082.709782,1067.499291,0.985951,1.0,1.0,1.0,1.0,1.0
8,1220.247903,2022,1152.677892,1137.069651,0.986459,1.0,1.0,1.0,1.0,1.0
9,1268.701746,2023,1209.330152,1193.333158,0.986772,1.0,1.0,1.0,1.0,1.0


## Pass changes to Tax-Calculator

In [90]:
indiv_rev_impact = distribute_results(iit_params_ref)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [91]:
for x in indiv_rev_impact:
    print x
print sum(indiv_rev_impact[3:])

0.0
1.78379219484e-15
0.0
-3.50207367584
-3.94591561466
-4.23266160549
-5.29666523272
-5.183134952
-5.1652162445
-5.22552487176
-5.34168924772
-5.47339428577
-5.66879766578
-5.87782632045
-54.9128997167


In [92]:
ModelResults = pd.DataFrame({'year': range(2014,2028), 'IndivTaxRev': indiv_rev_impact})
ModelResults['CorpTaxRev'] = combined_ref['taxrev'] - combined_base['taxrev']
ModelResults['RevenueChange'] = ModelResults['IndivTaxRev'] + ModelResults['CorpTaxRev']
ModelResults

,IndivTaxRev,year,CorpTaxRev,RevenueChange
0,0.000000e+00,2014,0.000000e+00,0.000000e+00
1,1.783792e-15,2015,5.684342e-14,5.862721e-14
2,0.000000e+00,2016,-1.136868e-13,-1.136868e-13
3,-3.502074e+00,2017,1.261565e+01,9.113579e+00
4,-3.945916e+00,2018,1.297659e+01,9.030670e+00
5,-4.232662e+00,2019,1.364250e+01,9.409837e+00
6,-5.296665e+00,2020,1.496127e+01,9.664602e+00
7,-5.183135e+00,2021,1.521049e+01,1.002736e+01
8,-5.165216e+00,2022,1.560824e+01,1.044302e+01
9,-5.225525e+00,2023,1.599699e+01,1.077147e+01


In [93]:
for i in range(3,14):
    print ModelResults['RevenueChange'][i]
print '\n'
print sum(ModelResults['RevenueChange'][3:13])

9.1135794448
9.03067001523
9.40983651556
9.6646021672
10.027355406
10.4430245011
10.7714689058
11.1473259373
11.5497170048
11.9279430196
12.3546896875


103.085522917
